In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import tensorflow as tf

from konlpy.tag import Mecab

import re
import os
import io
import time
import random

import seaborn as sns

%config InlineBackend.figure_format = 'retina'
 
sns.set(font='NanumGothic')

## Step 1. 데이터 다운로드

In [2]:
with open('korean-english-park.train.ko', 'r') as f:
    raw_ko = f.read().splitlines()
    
with open('korean-english-park.train.en', 'r') as f:
    raw_en = f.read().splitlines()
    
for sen in raw_ko[0:100][::20]: print(">>", sen)
print()
for sen in raw_en[0:100][::20]: print(">>", sen)

>> 개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"
>> 북한의 핵무기 계획을 포기하도록 하려는 압력이 거세지고 있는 가운데, 일본과 북한의 외교관들이 외교 관계를 정상화하려는 회담을 재개했다.
>> "경호 로보트가 침입자나 화재를 탐지하기 위해서 개인적으로, 그리고 전문적으로 사용되고 있습니다."
>> 수자원부 당국은 논란이 되고 있고, 막대한 비용이 드는 이 사업에 대해 내년에 건설을 시작할 계획이다.
>> 또한 근력 운동은 활발하게 걷는 것이나 최소한 20분 동안 뛰는 것과 같은 유산소 활동에서 얻는 운동 효과를 심장과 폐에 주지 않기 때문에, 연구학자들은 근력 운동이 심장에 큰 영향을 미치는지 여부에 대해 논쟁을 해왔다.

>> Much of personal computing is about "can you top this?"
>> Amid mounting pressure on North Korea to abandon its nuclear weapons program Japanese and North Korean diplomats have resumed talks on normalizing diplomatic relations.
>> “Guard robots are used privately and professionally to detect intruders or fire,” Karlsson said.
>> Authorities from the Water Resources Ministry plan to begin construction next year on the controversial and hugely expensive project.
>> Researchers also have debated whether weight-training has a big impact on the heart, since it does not give the heart and lungs the kind of workout they get from

## Step 2. 데이터 정제

1. set 데이터형이 중복을 허용하지 않는다는 것을 활용해 중복된 데이터를 제거하도록 합니다. 데이터의 병렬 쌍이 흐트러지지 않게 주의하세요! 중복을 제거한 데이터를 cleaned_corpus 에 저장합니다.  

2. 앞서 정의한 preprocessing() 함수는 한글에서는 동작하지 않습니다. 한글에 적용할 수 있는 정규식을 추가하여 함수를 재정의하세요!  
  
3. 타겟 언어인 영문엔 <start> 토큰과 <end> 토큰을 추가하고 split() 함수를 이용하여 토큰화합니다. 한글 토큰화는 KoNLPy의 mecab 클래스를 사용합니다.  
  
모든 데이터를 사용할 경우 학습에 굉장히 오랜 시간이 걸립니다. cleaned_corpus로부터 토큰의 길이가 40 이하인 데이터를 선별하여 eng_corpus와 kor_corpus를 각각 구축하세요.

In [3]:
df = pd.DataFrame({
    'ko' : raw_ko,
    'en' : raw_en
})

In [4]:
print('before dropping duplicates : ', df.shape)
df.drop_duplicates(['ko'], inplace=True, ignore_index=True)
df.drop_duplicates(['en'], inplace=True, ignore_index=True)
print('after dropping duplicates : ', df.shape)

before dropping duplicates :  (94123, 2)
after dropping duplicates :  (74849, 2)


In [5]:
# 결측치는 없음
df.dropna(how='any', inplace=True)
df.shape

(74849, 2)

In [6]:
df.shape

(74849, 2)

In [7]:
def preprocess_korean_sentence(sentence, s_token=False, e_token=False):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^가-힣?.!,]+", " ", sentence)

    sentence = sentence.strip()
    
    return sentence

In [8]:
def preprocess_english_sentence(sentence, s_token=True, e_token=True):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()

    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    return sentence

In [9]:
enc_corpus = []
dec_corpus = []

for pair in zip(df['ko'], df['en']):
    (ko, en) = pair
    
    enc_corpus.append(preprocess_korean_sentence(ko))
    dec_corpus.append(preprocess_english_sentence(en, s_token=True, e_token=True))

In [10]:
enc_corpus[:10]

['개인용 컴퓨터 사용의 상당 부분은 이것보다 뛰어날 수 있느냐 ?',
 '모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다 .',
 '그러나 이것은 또한 책상도 필요로 하지 않는다 .',
 '. 달러하는 이 최첨단 무선 광마우스는 허공에서 팔목 , 팔 , 그외에 어떤 부분이든 그 움직임에따라 커서의 움직임을 조절하는 회전 운동 센서를 사용하고 있다 .',
 '정보 관리들은 동남 아시아에서의 선박들에 대한 많은 테러 계획들이 실패로 돌아갔음을 밝혔으며 , 세계 해상 교역량의 거의 분의 을 운송하는 좁은 해로인 말라카 해협이 테러 공격을 당하기 쉽다고 경고하고 있다 .',
 '이 지역에 있는 미국 선박과 상업용 선박들에 대한 알카에다의 테러 시도 중 여러 건이 실패했다는 것을 알게 된 후에 , 전문가들은 테러 조직이 여전히 세계 경제에 타격을 입히려 한다고 경고하고 있으며 , 동남 아시아에 있는 세계 경제의 주요 통로가 위험에 처해 있다고 그들은 생각하고 있다 .',
 '국립 과학 학회가 발표한 새 보고서에따르면 , 복잡한 임무를 수행해야 하는 군인들이나 보다 오랜 시간 동안 경계를 늦추지 않고 있기 위해 도움이 필요한 군인들에게 카페인이 반응 시간을 증가시키고 임무 수행 능력을 향상시키는데 도움이 된다고 한다 .',
 '이 보고서에따르면 , 특히 , 군사 작전에서 생사가 걸린 상황이 될 수도 있는 반응 속도와 시각 및 청각의 경계 상태를 유지시키기 위해 카페인이 사용될 수도 있다 . 고 한다 .',
 '결정적인 순간에 그들의 능력을 증가시켜 줄 그 무엇이 매우 중요합니다 .',
 '연구가들이 이미 커피 대체품으로서 음식 대용 과자나 껌에 카페인을 첨가하는 방법을 연구하고 있다고 는 말했다 .']

In [11]:
dec_corpus[:10]

['<start> much of personal computing is about can you top this ? <end>',
 '<start> so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable , wireless mouse . <end>',
 '<start> like all optical mice , but it also doesn t need a desk . <end>',
 '<start> uses gyroscopic sensors to control the cursor movement as you move your wrist , arm , whatever through the air . <end>',
 '<start> intelligence officials have revealed a spate of foiled plots on ships in southeast asia and are warning that a narrow stretch of water carrying almost one third of the world s maritime trade is vulnerable to a terror attack . <end>',
 '<start> after learning of several foiled al qaeda attempts on u . s . and commercial ships in the area , experts are warning that the terror network still wants to cripple the global economy , the world s economic jugular vein in southeast asia is at risk . <end>',
 '<start> caffeine can help increase reaction time and improve per

In [12]:
enc_corpus = [ x for x in enc_corpus if len(x.split())<=40]

In [13]:
dec_corpus = [ x for x in dec_corpus if len(x.split())<=40]

## Step 3. 데이터 토큰화

앞서 정의한 tokenize() 함수를 사용해 데이터를 텐서로 변환하고 각각의 tokenizer를 얻으세요! 단어의 수는 실험을 통해 적당한 값을 맞춰주도록 합니다! (최소 10,000 이상!)  
  
❗ 주의: 난이도에 비해 데이터가 많지 않아 훈련 데이터와 검증 데이터를 따로 나누지는 않습니다.  

In [14]:
def get_mecab_tokenize(data):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    m = Mecab()
    
    mecab_corpus = []
    
    max_len = 40
    
    for sentence in data:
        mecab_corpus.append(m.morphs(sentence))

    tokenizer.fit_on_texts(mecab_corpus)
    mecab_tensor = tokenizer.texts_to_sequences(mecab_corpus)
    mecab_tensor = tf.keras.preprocessing.sequence.pad_sequences(mecab_tensor, padding='post', maxlen=max_len)
    return mecab_tensor, tokenizer

In [15]:
enc_tensor, enc_tokenizer = get_mecab_tokenize(enc_corpus)

In [16]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    return tensor, tokenizer

In [17]:
dec_tensor, dec_tokenizer = tokenize(dec_corpus)

In [18]:
dec_tensor[:10]

array([[    4,   265,     7,  1229,  7110,    16,    42,    92,    86,
          204,    40,   221,     5,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [    4,   133,     8,  4122,     8,   373,   361,   268,    42,
            8, 14496,  3346, 16657,  5466,   955,     9,   177, 14496,
            3,  3346,  5466,     2,     5,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [    4,   156,    76, 16657,  4123,     3,    32,    19,    51,
          976,    77,   448,     8,  6819,     2,     5,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [    4,  2932, 26114,  7821,  

In [19]:
len(dec_tensor)

66407

In [20]:
len(enc_tensor)

74641

In [21]:
enc_tensor = enc_tensor[:10000]
dec_tensor = dec_tensor[:10000]

## Step 4. 모델 설계

한국어를 영어로 잘 번역해 줄 멋진 Attention 기반 Seq2seq 모델을 설계하세요!  
앞서 만든 모델에 Dropout 모듈을 추가하면 성능이 더 좋아집니다!     
Embedding Size와 Hidden Size는 실험을 통해 적당한 값을 맞춰 주도록 합니다!    

In [22]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units)
        self.w_enc = tf.keras.layers.Dense(units)
        self.w_com = tf.keras.layers.Dense(1)
    
    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]

        h_enc = self.w_enc(h_enc)
        h_dec = tf.expand_dims(h_dec, 1)
        h_dec = self.w_dec(h_dec)

        score = self.w_com(tf.nn.tanh(h_dec + h_enc))
        
        attn = tf.nn.softmax(score, axis=1)

        context_vec = attn * h_enc
        context_vec = tf.reduce_sum(context_vec, axis=1)

        return context_vec, attn

In [23]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(enc_units,
                                       return_sequences=True)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.gru(out)
        
        return out

In [24]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units,
                                       return_sequences=True,
                                       return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, h_dec, enc_out):
        context_vec, attn = self.attention(enc_out, h_dec)

        out = self.embedding(x)
        out = tf.concat([tf.expand_dims(context_vec, 1), out], axis=-1)
        
        out, h_dec = self.gru(out)
        out = tf.reshape(out, (-1, out.shape[2]))
        out = self.fc(out)

        return out, h_dec, attn

In [25]:
BATCH_SIZE     = 32
SRC_VOCAB_SIZE = len(enc_tokenizer.index_word) + 1
TGT_VOCAB_SIZE = len(dec_tokenizer.index_word) + 1

units         = 1024
embedding_dim = 512

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)

# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print ('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                     sample_state, sample_output)

print ('Decoder Output:', sample_logits.shape)
print ('Decoder Hidden State:', h_dec.shape)
print ('Attention:', attn.shape)

Encoder Output: (32, 30, 1024)
Decoder Output: (32, 40948)
Decoder Hidden State: (32, 1024)
Attention: (32, 30, 1)


In [26]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    
    return tf.reduce_mean(loss)

## Step 5. 훈련하기

훈련엔 위에서 사용한 코드를 그대로 사용하되, eval_step() 부분이 없음에 유의합니다! 매 스텝 아래의 예문에 대한 번역을 생성하여 본인이 생각하기에 가장 멋지게 번역한 Case를 제출하세요! (Attention Map을 시각화해보는 것도 재밌을 거예요!)

In [27]:
@tf.function
def train_step(src, tgt, encoder, decoder, optimizer, dec_tok):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape:
        enc_out = encoder(src)
        h_dec = enc_out[:, -1]
        
        dec_src = tf.expand_dims([dec_tok.word_index['<start>']] * bsz, 1)

        for t in range(1, tgt.shape[1]):
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

            loss += loss_function(tgt[:, t], pred)
            dec_src = tf.expand_dims(tgt[:, t], 1)
            
    batch_loss = (loss / int(tgt.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

In [28]:
tf.keras.backend.clear_session()

In [ ]:
# Training Process

from tqdm import tqdm

EPOCHS = 2

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_tensor.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss = train_step(enc_tensor[idx:idx+BATCH_SIZE],
                                dec_tensor[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                dec_tokenizer)
    
        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

Epoch  2:  29%|██▉       | 91/313 [00:38<01:36,  2.30it/s, Loss 4.0434]

## 회고

**배운점**
  
* 성능이 떨어지지만 단순 번역기를 만들어보았다.
  
**아쉬운점**
  
* 어제 결석으로 짧은시간 동안 실습하느라 최종 번역결과를 확인하지 못했다.
  
**느낀점**

* 데이터 변환이나 전처리, 토크나이징 등 기본적인 기술에 대해 계속해서 까먹는다. 반복 숙달이 필요하다.

**어려웠던 점**

* Training 하는 시간이 길고 GPU 용량 문제가 있었다.
* 토크나이징을 여러번 해봤는데 새삼스럽게 데이터 처리가 헷갈렸다.